### Import Required Libraries and Set Up Environment Variables

In [49]:
# Dependencies
from urllib.parse import urlencode
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import time
import os

In [50]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

print("NTY API Loaded:", type(nyt_api_key))
print("Movie DB API Loaded:", type(tmdb_api_key))

NTY API Loaded: <class 'str'>
Movie DB API Loaded: <class 'str'>


### Access the New York Times API

In [51]:
# Set the base URL
url = "https://api.nytimes.com"
endpoint = "/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

query_params = {
    "begin_date": begin_date,
    "end_date": end_date,
    "fl": field_list,
    "fq": filter_query,
    "sort": sort,
    "api-key": nyt_api_key
}

encoded_params = urlencode(query_params)

# Build URL
full_url = (
    f"{url}"
    f"{endpoint}"
    f"{encoded_params}"
)

# Print URL (Hiding API Key)
print("Full URL:", full_url.replace(nyt_api_key, "<API_KEY>"))

Full URL: https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20130101&end_date=20230531&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&fq=section_name%3A%22Movies%22+AND+type_of_material%3A%22Review%22+AND+headline%3A%22love%22&sort=newest&api-key=<API_KEY>


In [52]:
# Create an empty list to store the reviews
reviews_list = []

#! loop through pages 0-19
for page_index in range(0, 1):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{full_url}&page={page_index}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews_data = response.json()

    #! Add a twelve second interval between queries to stay within API query limits
    time.sleep(0.1)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews_data["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Page {page_index} retrieved successfully!")

    except KeyError as error:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page_index}.", f"Error: {error}")

Page 0 retrieved successfully!


In [53]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [54]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

print("Shape:", reviews_df.shape)
reviews_df

Shape: (10, 16)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}]",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Kapur, Shekhar', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'James, Lily', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Azmi, Shabana', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'Thompson, Emma', 'rank': 5, 'major': 'N'}]",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'You Can Live Forever (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Slutsky, Mark', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Watts, Sarah (Film Director)', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'O'Driscoll, Anwen', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Laporte, June (Actor)', 'rank': 6, 'major': 'N'}]",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'A Tourist's Guide to Love (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Tsuchida, Steven', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Cook, Rachael Leigh', 'rank': 4, 'major': 'N'}]",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New Yo

In [55]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  10 non-null     object
 1   snippet                  10 non-null     object
 2   source                   10 non-null     object
 3   keywords                 10 non-null     object
 4   pub_date                 10 non-null     object
 5   word_count               10 non-null     int64 
 6   headline.main            10 non-null     object
 7   headline.kicker          2 non-null      object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  10 non-null     object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null      object
 13  byline.original          10 non-null     object
 14  byline.person            10 non-null     obje

In [56]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

print("Shape:", reviews_df.shape)
reviews_df

Shape: (10, 17)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}]",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Kapur, Shekhar', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'James, Lily', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Azmi, Shabana', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'Thompson, Emma', 'rank': 5, 'major': 'N'}]",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'You Can Live Forever (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Slutsky, Mark', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Watts, Sarah (Film Director)', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'O'Driscoll, Anwen', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Laporte, June (Actor)', 'rank': 6, 'major': 'N'}]",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'A Tourist's Guide to Love (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Tsuchida, Steven', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Cook, Rachael Leigh', 'rank': 4, 'major': 'N'}]",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as

In [57]:
reviews_df['title'].values

array(['The Attachment Diaries',
       'What’s Love Got to Do With It?’ Probably a Lo',
       'You Can Live Forever', 'A Tourist’s Guide to Love',
       'Other People’s Children', 'One True Loves',
       'The Lost Weekend: A Love Story', 'A Thousand and One',
       'Your Place or Mine', 'Love in the Time of Fentanyl'], dtype=object)

In [58]:
reviews_df['headline.main'].values

array(['‘The Attachment Diaries’ Review: Love, Sick',
       'Review: ‘What’s Love Got to Do With It?’ Probably a Lot',
       '‘You Can Live Forever’ Review: Do You Love Me Now?',
       '‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip',
       '‘Other People’s Children’ Review: True Romance',
       '‘One True Loves’ Review: A Romance Lost at Sea',
       '‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed',
       '‘A Thousand and One’ Review: A New York Love Story',
       '‘Your Place or Mine’ Review: Try Neither',
       '‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouver'],
      dtype=object)

In [59]:
reviews_df['keywords'].values[0]

[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
 {'name': 'creative_works',
  'value': 'The Attachment Diaries (Movie)',
  'rank': 2,
  'major': 'N'},
 {'name': 'persons',
  'value': 'Diment, Valentin Javier',
  'rank': 3,
  'major': 'N'}]

In [60]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df["keywords"].apply(extract_keywords)

In [61]:
reviews_df['keywords'].values[0]

'subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;'

In [62]:
print("Shape:", reviews_df.shape)
reviews_df

Shape: (10, 17)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins

In [74]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()

titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl']

### Access The Movie Database API

In [64]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [65]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



In [66]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [67]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [68]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [69]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [70]:
# Drop "byline.person" column


In [71]:
# Delete duplicate rows and reset index


In [72]:
# Export data to CSV without the index
